In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.7 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from transformers import DebertaModel, DebertaTokenizerFast, AdamW, get_linear_schedule_with_warmup
from tqdm.auto import tqdm

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving prompts_test.csv to prompts_test (4).csv
Saving prompts_train.csv to prompts_train (4).csv
Saving sample_submission.csv to sample_submission (4).csv
Saving summaries_test.csv to summaries_test (4).csv
Saving summaries_train.csv to summaries_train (4).csv


1. Data cleaning and preprocessing



In [ ]:
# load the required files
train_prompts = pd.read_csv('prompts_train.csv')
train_summaries = pd.read_csv('summaries_train.csv')

# merging the two dataframes to get every data for each text
df = pd.merge(train_prompts, train_summaries, on="prompt_id")

# cleaning and removing unnecessary strings in prompt text
new_prompt_text = []

for i in df['prompt_text']:
  new_prompt_text.append(i.replace('\r\n', ''))

df['prompt_text'] = new_prompt_text
df


,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058
2,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,0094589c7a22,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181
3,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,00cd5736026a,One element of an Ideal tragedy is having a co...,0.088882,-0.594710
4,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,00d98b8ff756,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886
5,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,00db6d2b9222,"Tragedy much b complex, Tragedy much b complex...",-1.547163,-1.461245
6,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,0106a4b99776,One element of a perfect tragery in Aristotle'...,1.199642,1.078906
7,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,011acba6c31c,It should have a character that people can rel...,-1.264214,-1.505073
8,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,014da834aab7,A tragedy should feature a character who can b...,0.670351,1.551633
9,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 As the sequel to what has already b...,016714bf867a,"In the second paragraph Aritole explained, ""A ...",-1.547163,-1.461245


The code here is to create a dataset class for our raw data.

# **1. Albert testing**

In [ ]:
class CommonLitDataSet(Dataset):

    def __init__(self, text, prompt, targets, tokenizer):
      # take in text, prompt_question, [content, wording], and the tokenizer
        self.text = text
        self.prompt = prompt
        self.targets = targets
        self.tokenizer = tokenizer
        self.sep_token = self.tokenizer.sep_token if self.tokenizer.sep_token else '[SEP]'

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = self.text[idx]
        prompt = self.prompt[idx]
        target = self.targets[idx]

        # concatenate prompt and text
        prompt_text = f'{prompt} {self.sep_token} {text}'

        encoding = self.tokenizer(prompt_text, truncation=True, padding='max_length', max_length=512)

        inputs = {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long),
            'targets': torch.tensor(target, dtype=torch.float)
        }

        return inputs




Here is the code for the model (which returns 2 results)

In [ ]:
from transformers import AutoModel, AutoTokenizer, AutoConfig

class Model(nn.Module):
    def __init__(self, model_name):
        super().__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # load the model
        self.model = AutoModel.from_pretrained(model_name)

        # use a linear layer to reduce the dimension of the output
        self.reduce_dim = nn.Linear(self.model.config.hidden_size, 256)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(256, 2)  # Assuming this is a regression task

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        output = output.last_hidden_state.mean(dim=1)
        output = self.activation(self.reduce_dim(output))
        output = self.dropout(output)
        logits = self.linear(output)
        return logits

    def predict(self, dataloader):
        self.eval()  # set the model to eval mode
        preds = []

        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(device) # Adds an extra dimension
                attention_mask = batch['attention_mask'].to(device) # Adds an extra dimension
                outputs = self(input_ids, attention_mask)
                preds.append(outputs.detach().cpu())

        return torch.cat(preds, dim=0)





In [ ]:
# define the metrics
def MCRMSE(targets, outputs):
    colwise_mse = torch.mean((targets - outputs) ** 2, axis=0)
    return torch.mean(torch.sqrt(colwise_mse))

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    best_loss = np.inf
    patience = 2
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        model.train()
        epoch_loss = 0
        num_batches = 0
        train_loader = tqdm(train_loader)

        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target = batch['targets'].to(device)
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()
            num_batches += 1

            train_loader.set_description(f"Epoch {epoch+1}/{num_epochs}")

        print(f'Epoch {epoch+1}, Loss: {epoch_loss/num_batches}')
        val_loss = 0
        num_val_batches = 0
        model.eval()

        with torch.no_grad():
            for val_batch in val_loader:
                input_ids = val_batch['input_ids'].to(device)
                attention_mask = val_batch['attention_mask'].to(device)
                target = val_batch['targets'].to(device)
                # print(input_ids)
                # print(attention_mask)
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, target)
                val_loss += loss.item()
                num_val_batches += 1

        avg_val_loss = val_loss/num_val_batches
        print(f"End of Epoch {epoch+1}/{num_epochs}, Validation loss: {avg_val_loss}")

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping!')
                break

    return model


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np
import torch

class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, model_names, meta_model, models_directory):
        self.model_names = model_names
        self.meta_model = meta_model
        self.models_directory = models_directory
        self.kf = KFold(n_splits=4, random_state=42, shuffle=True)

    def fit(self, X, y):
        self.saved_models = []
        oof_train_wording = np.zeros((X.shape[0], len(self.model_names)))
        oof_train_content = np.zeros((X.shape[0], len(self.model_names)))

        for i, model_name in enumerate(self.model_names):
            print(f"Training model: {model_name}")
            model_list = []
            cv_scores = []
            for train_index, val_index in self.kf.split(X,y):
                print(train_index, val_index)
                model = Model(model_name).to(device)
                model_list.append(model)
                # Prepare the data
                text_train, text_val = X['text'].iloc[train_index].reset_index(drop=True), X['text'].iloc[val_index].reset_index(drop=True)
                prompt_train, prompt_val = X['prompt_question'].iloc[train_index].reset_index(drop=True), X['prompt_question'].iloc[val_index].reset_index(drop=True)
                y_train, y_val = y.iloc[train_index].reset_index(drop=True), y.iloc[val_index].reset_index(drop=True)
                # Create DataLoaders
                tokenizer = AutoTokenizer.from_pretrained(model_name)
                train_dataset = CommonLitDataSet(text_train, prompt_train, y_train.values, tokenizer)
                train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
                val_dataset = CommonLitDataSet(text_val, prompt_val, y_val.values, tokenizer)
                val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
                optimizer = AdamW(model.parameters(), lr=1e-5)
                criterion = MCRMSE
                num_epochs = 8
                # Train the model
                trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device)
                model_list[-1] = trained_model
                predictions = trained_model.predict(val_loader)
                oof_train_wording[val_index, i] = predictions[:, 0]
                oof_train_content[val_index, i] = predictions[:, 1]
                cv_scores.append(criterion(predictions, y_val).item())
            self.saved_models.append(model_list)

        oof_train = np.hstack((oof_train_wording, oof_train_content))
        self.meta_model.fit(oof_train, y)
        return self

    def predict(self, X):
        final_predictions_wording = []
        final_predictions_content = []

        for i, model_list in enumerate(self.saved_models):
            model_name = self.model_names[i]
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            test_dataset = CommonLitDataSet(X['text'], X['prompt_question'], np.zeros(len(X)), tokenizer)
            test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
            model_predictions = np.column_stack([model.predict(test_loader).numpy() for model in model_list])
            final_predictions_wording.append(model_predictions[:, 0])
            final_predictions_content.append(model_predictions[:, 1])

        print(final_predictions_wording, final_predictions_content)

        meta_features = np.hstack((np.column_stack(final_predictions_wording), np.column_stack(final_predictions_content)))
        return self.meta_model.predict(meta_features)


In [ ]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_names = ['microsoft/deberta-base', 'albert-base-v2', 'bert-base-uncased']

models_directory = "/content/gdrive/My Drive"

stack_model = stacking(model_names = model_names,
                       meta_model=MultiOutputRegressor(SVR(kernel='linear')),
                       models_directory = models_directory)

x = df[['text', 'prompt_question']]
y = df[['content', 'wording']]

stack_model.fit(x, y)



Training model: microsoft/deberta-base
[ 1  2  7 10 11 14 16 18 20 21 22 23 24 26 28 29 32 35 37 38 39 41 42 44
 47 49 51 53 55 59] [ 0  3  4  5  6  8  9 12 13 15 17 19 25 27 30 31 33 34 36 40 43 45 46 48
 50 52 54 56 57 58]
Epoch 1/1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1, Loss: 0.7643040719131629
End of Epoch 1/1, Validation loss: 0.818664891521136
microsoft_deberta-base


RuntimeError: ignored

In [ ]:
stack_model.predict(dc[['text', 'prompt_question']])

NotFittedError: ignored

In [ ]:
from joblib import dump

# save the stacking model to the desired path
model_save_name = 'stack_model.joblib'
path = "/content/gdrive/My Drive" + model_save_name

dump(stack_model, path)
